#### Import necessary libraries:

In [1]:
import boto3
import time 
import pandas as pd
import io
import configparser

#### Configure aws services:

In [2]:
config = configparser.ConfigParser()
config.read_file(open('aws.config'))

In [3]:
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')
AWS_REGION = config.get('AWS', 'AWS_REGION')

SCHEMA_NAME = config.get('ATHENA', 'SCHEMA_NAME')
S3_STAGING_DIR = config.get('S3', 'S3_STAGING_DIR')
S3_BUCKET_NAME = config.get('S3', 'S3_BUCKET_NAME')
S3_OUTPUT_DIR = config.get('S3', 'S3_OUTPUT_DIR')


#### Connecting to Athena and Query data:

In [4]:
athena_client = boto3.client('athena', aws_access_key_id=KEY, aws_secret_access_key=SECRET, region_name=AWS_REGION)

In [5]:
Dict = {}
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            # This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(15)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client("s3", aws_access_key_id=KEY, aws_secret_access_key=SECRET, region_name=AWS_REGION)
    s3_client.download_file(S3_BUCKET_NAME, f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv", temp_file_location,)
    return pd.read_csv(temp_file_location)


##### enigma_jhud table:

In [6]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
enigma_jhud_df = download_and_load_query_results(athena_client, response)
enigma_jhud_df.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


#### enigma_nytimes_data_in_usa_us_county

In [7]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa_us_county LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
enigma_nytimes_data_in_usa_us_county_df = download_and_load_query_results(athena_client, response)
enigma_nytimes_data_in_usa_us_county_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


##### enigma_nytimes_data_in_usa_us_states

In [8]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa_us_states LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
enigma_nytimes_data_in_usa_us_states_df = download_and_load_query_results(athena_client, response)
enigma_nytimes_data_in_usa_us_states_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


##### rearc_covid_19_testing_data_states_daily

In [9]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_daily LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_covid_19_testing_data_states_daily_df = download_and_load_query_results(athena_client, response)
rearc_covid_19_testing_data_states_daily_df.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


##### rearc_covid_19_testing_data_us_total_latest

In [10]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_total_latest LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_covid_19_testing_data_us_total_latest_df = download_and_load_query_results(athena_client, response)
rearc_covid_19_testing_data_us_total_latest_df.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


##### rearc_usa_hospital_beds

In [11]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
rearc_usa_hospital_beds_df = download_and_load_query_results(athena_client, response)
rearc_usa_hospital_beds_df.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129,129,0,0,NaN,NaN,0.0,0,33.495498,-112.066157
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295,295,2,2,NaN,NaN,2.0,0,32.181263,-110.965885
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57,57,2,2,NaN,NaN,2.0,0,36.773324,-119.779742
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216,216,1,1,NaN,NaN,2.0,0,41.284400,-72.957610
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60,60,0,0,NaN,NaN,1.0,0,39.740206,-75.606532


In [12]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countrycode LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_countrycode_df = download_and_load_query_results(athena_client, response)
static_datasets_countrycode_df.head()

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [13]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countypopulation LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_countypopulation_df = download_and_load_query_results(athena_client, response)
static_datasets_countypopulation_df.head()

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


##### static_datasets_state_abv

In [14]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_state_abv LIMIT 10",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
static_datasets_state_abv_df = download_and_load_query_results(athena_client, response)
static_datasets_state_abv_df.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [15]:
new_header = static_datasets_state_abv_df.iloc[0]
static_datasets_state_abv_df = static_datasets_state_abv_df[1:]
static_datasets_state_abv_df.columns = new_header
static_datasets_state_abv_df.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA
